In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import AutoConfig
# toke_config = AutoConfig.from_pretrained("Pollawat/mt5-small-thai-qa-qg")
# toke_config.vocab_size


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig, AutoModel,MT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("Pollawat/mt5-small-thai-qa-qg")
# m_config = AutoConfig.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap", vocab_size=toke_config.vocab_size)
model = AutoModelForSeq2SeqLM.from_pretrained("Pollawat/mt5-small-thai-qa-qg")

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [6]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

In [7]:
# from transformers import MT5ForConditionalGeneration
# m2 = MT5ForConditionalGeneration.from_pretrained("Pollawat/mt5-small-thai-qa-qg")
# m2

In [8]:
import tensorflow as tf

In [9]:
import os
import numpy as np
import pandas as pd
import transformers
import torch
from transformers import TFAutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [10]:
from datasets import load_dataset

dataset = load_dataset("iapp_wiki_qa_squad")

Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
train_dataset  = load_dataset('iapp_wiki_qa_squad', split="train")
valid_dataset = load_dataset('iapp_wiki_qa_squad', split="validation")
test_dataset = load_dataset("iapp_wiki_qa_squad", split="test")

Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


In [12]:
# train_dataset[0]["question"] = train_dataset["question"][0] + "asd"

In [13]:
# train_dataset["question"][0] = train_dataset["question"][0][:train_dataset["question"][0].find("เท่าไร")+len("เท่าไร")] + "?" + train_dataset["question"][0][train_dataset["question"][0].find("เท่าไร")+len("เท่าไร"):]

In [14]:
train_dataset["question"][0]

'พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร'

In [15]:
q_words = ["ใคร","ทำไม", "อย่างไร", "ที่ไหน", "อะไร", "เหตุใด", "หรือไม่", "เท่าไร", "เท่าใด", "ที่ไหน", "ที่ใด", "หรือไม่", "ยังไง", "ไหน", "เมื่อไร", "รึยัง", "ใช่ไหม","เมื่อใด","ใด","กี่", "ไหร่"]

In [16]:
def add_q_ques(dataset):
  if "?" not in dataset["question"]:
    dataset["question"] = dataset["question"] + " ?"
  # for i in q_words:
    
    # if "?" in dataset["question"]:
    #   break
    # if i in dataset["question"]:
    #   dataset["question"] = dataset["question"][:dataset["question"].find(i)+len(i)] + "?" + dataset["question"][dataset["question"].find(i)+len(i):]
    #   break
  return dataset

In [17]:
train_dataset = train_dataset.map(add_q_ques)

Loading cached processed dataset at /root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3/cache-9611e926a535c43e.arrow


In [18]:
def edit_ans_to_ds(dataset):
#     ans_list = [ans_dict["text"][0] for ans_dict in dataset["answers"]]
    dataset["ans_list"] = dataset["answers"]["text"][0]
#     dataset.add_column("ans_list", ans_list)
    return dataset

In [19]:
train_dataset = train_dataset.map(edit_ans_to_ds)
train_dataset

  0%|          | 0/5761 [00:00<?, ?ex/s]

Dataset({
    features: ['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'],
    num_rows: 5761
})

In [20]:
def create_ans_context(dataset):
    answer = dataset["ans_list"]
    question = dataset["question"]
    # input_format = tokenizer("answer: %s  context: %s </s>" % (answer, context), truncation=True ,padding="max_length", max_length=256 )
    input_format = tokenizer(dataset["context"], padding="max_length", truncation=True, max_length=512)
    # labels = tokenizer("<pad> <extra_id_0> %s <ANS> %s" % (question, answer))
    labels = tokenizer("%s <ANS> %s" % (question, answer))
    # labels = tokenizer(dataset["question"],truncation=True ,padding="max_length", max_length=256)
    dataset["input_ids"] = input_format["input_ids"]
    dataset["attention_mask"] = input_format["attention_mask"]
    dataset["labels"] = labels["input_ids"]
    return dataset

In [21]:
train_ds_processed = train_dataset.map(create_ans_context)

  0%|          | 0/5761 [00:00<?, ?ex/s]

In [22]:
train_ds_processed = train_ds_processed.remove_columns(['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'])

In [23]:
train_ds_processed

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5761
})

In [24]:
train_ds_processed.set_format("torch")

In [25]:
tokenizer.decode(train_ds_processed[0]["labels"])

'พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร? <ANS> 3 ธันวาคม พ.ศ. 2533</s>'

In [26]:
train_ds_processed[0]["labels"]

tensor([ 16912, 169164,  27987,  35872, 180094,  30007,    259,  17958, 108604,
         39322,   9211,   5095,  17958,  35872,  53477,  52854,   4215, 136233,
           259,  35749,  65194, 235473,    259,    291,    259, 250100,    381,
           259, 108341,  16912,    260,  17958,    260,    259, 235490,      1])

In [27]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_ds_processed, shuffle=True, batch_size=1)

In [37]:
# next(iter(train_dataloader))

In [28]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

In [29]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [39]:
tokenizer.decode(train_ds_processed[2]["labels"])

'พัทธ์ธีรา ศรุติพงศ์โภคิน ทําอาชีพอะไร? <ANS> นักแสดงหญิงชาวไทย</s>'

In [46]:
data_test_output = next(iter(train_dataloader))

In [45]:
# data_test_output

In [32]:
output = model.generate(data_test_output["input_ids"])

In [33]:
tokenizer.decode(output[0])

'<pad> <extra_id_0> วอลเลย์บอลหญิงชิงแชมป์นอร์เซกา 1973 จัดขึ้นที่ใด<ANS> '

In [47]:
tokenizer.decode(data_test_output["labels"][0])

'ท่าอากาศยานนาฮะ ตั้งอยู่ในจังหวัดอะไร? <ANS> จังหวัดโอกินาวะ</s>'

In [ ]:
# 11 15 18

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (w

In [ ]:
# model.from_pretrained("/content/drive/MyDrive/weight/Pollawat_PollawatTrainStyle")

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        running_loss += loss.item()
    # if epoch%3 == 0 and epoch != 0:
    model.save_pretrained("/content/drive/MyDrive/weight/Pollawat_PollawatTrainStyle")
    print(epoch, running_loss)

  0%|          | 0/28805 [00:00<?, ?it/s]

0 160051.63615715504


In [ ]:
print(running_loss)

In [ ]:
model.save_pretrained("/content/drive/MyDrive/weight/Pollawat_PollawatTrainStyle")

In [ ]:
num_training_steps

In [ ]:
x_i = next(iter(train_dataloader))

In [ ]:
tokenizer.decode(x_i["input_ids"][0])

In [ ]:
print("label:", tokenizer.decode(x_i["labels"][0]))

In [ ]:
b_i = {k: v.to(device) for k, v in x_i.items()}
# b_i

In [ ]:
# print(b_i["input_ids"])
out_2 = model.generate(input_ids=b_i["input_ids"],
               attention_mask=b_i["attention_mask"],
               num_beams=5)
print("Label", tokenizer.decode(b_i["labels"][0]))
print("ans", tokenizer.decode(out_2[0]))
print("................")
print(tokenizer.decode(b_i["input_ids"][0]))
print("")

In [ ]:
tokenizer.decode(out_2[0])

In [ ]:
out_2[0]